## <span style='background-color:#fff651'> Data Science Level 2 Practical Problem 2305-5 </span>

In [1]:
import pandas as pd
import numpy as np

purchase = pd.read_csv('lv2-2305-5.csv')
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30308 entries, 0 to 30307
Data columns (total 7 columns):
USER_ID       30308 non-null int64
PRD_CAT       30308 non-null int64
PRD_ID        30308 non-null object
GENDER        30308 non-null object
AGE_GRP       30308 non-null object
MARITAL_YN    30308 non-null int64
PUR_AMT       30308 non-null int64
dtypes: int64(4), object(3)
memory usage: 1.6+ MB


### <span style='background-color:#fff651'> Question 1 </span>

In [2]:
# Step 1-1
df1 = purchase.pivot_table(index=['MARITAL_YN','USER_ID'], columns='PRD_CAT', values='PUR_AMT', aggfunc=sum, fill_value=0).reset_index().rename_axis(None, axis=1)
df1.columns = ['MARITAL_YN','USER_ID','CAT1_AMT','CAT2_AMT','CAT3_AMT']
df1.head()

,MARITAL_YN,USER_ID,CAT1_AMT,CAT2_AMT,CAT3_AMT
0,0,1000003,162439,29054,10906
1,0,1000006,93457,35997,59314
2,0,1000009,78545,0,21602
3,0,1000011,58290,13852,2663
4,0,1000012,46368,12788,0


In [3]:
# Step 1-2
cor0 = df1[df1['MARITAL_YN']==0][['CAT1_AMT','CAT2_AMT','CAT3_AMT']].corr()
for i in [0,1,2]:
    cor0.iloc[i,i] = np.nan
cor0

,CAT1_AMT,CAT2_AMT,CAT3_AMT
CAT1_AMT,NaN,0.728646,0.553856
CAT2_AMT,0.728646,NaN,0.650382
CAT3_AMT,0.553856,0.650382,NaN


In [4]:
cor1 = df1[df1['MARITAL_YN']==1][['CAT1_AMT','CAT2_AMT','CAT3_AMT']].corr()
for i in [0,1,2]:
    cor1.iloc[i,i] = np.nan
cor1

,CAT1_AMT,CAT2_AMT,CAT3_AMT
CAT1_AMT,NaN,0.695264,0.509462
CAT2_AMT,0.695264,NaN,0.709962
CAT3_AMT,0.509462,0.709962,NaN


In [5]:
# Step 1-3
A = np.max(np.max(np.abs(cor0)))
B = np.min(np.min(np.abs(cor1)))
C = np.abs(A - B)
print('A:',A, ',B:',B, ',C:',C)

A: 0.7286459763376788 ,B: 0.5094616685957604 ,C: 0.21918430774191844


In [6]:
# Answer 1
ans21 = np.round(C, 3)
print(ans21)

0.219


### <span style='background-color:#fff651'> Question 2</span>

In [7]:
purchase

,USER_ID,PRD_CAT,PRD_ID,GENDER,AGE_GRP,MARITAL_YN,PUR_AMT
0,1001395,1,P00270942,M,26-35,1,11535
1,1001004,3,P00004842,M,26-35,0,5724
2,1000819,1,P00036842,M,36-45,0,11787
3,1001611,1,P00127742,M,26-35,0,8125
4,1000438,1,P00129342,M,18-25,0,19096
...,...,...,...,...,...,...,...
30303,1001449,3,P00294442,M,36-45,1,5612
30304,1001694,1,P00034442,M,26-35,1,15501
30305,1000889,1,P00249742,M,46-50,0,4020
30306,1000714,1,P00193542,M,18-25,0,19342


In [8]:
# Step 2-1
df2 = purchase.pivot_table(index=['PRD_ID'], columns='AGE_GRP', values='PUR_AMT', aggfunc=sum, fill_value=0).reset_index().rename_axis(None, axis=1)
df2.columns = ['PRD_ID','AGE1_AMT','AGE2_AMT','AGE3_AMT','AGE4_AMT','AGE5_AMT']
print(df2.shape)
df2.head()

(707, 6)


,PRD_ID,AGE1_AMT,AGE2_AMT,AGE3_AMT,AGE4_AMT,AGE5_AMT
0,P00000142,532314,857816,410458,110167,78328
1,P00000242,163997,254667,124015,88333,22685
2,P00000642,370517,574747,175068,47192,39675
3,P00000842,26105,0,0,0,0
4,P00000942,64393,51194,50653,0,0


In [9]:
# Step 2-2
from sklearn.preprocessing import StandardScaler 

df2z = pd.DataFrame(StandardScaler().fit_transform(df2.drop(columns='PRD_ID')))

In [10]:
# Step 2-3
from sklearn.cluster import KMeans

df2['CLST'] = KMeans(random_state=1234, n_clusters=3, init='random', n_init=50).fit_predict(df2z)

In [11]:
# Step 2-4
df2_agg = df2.groupby('CLST').mean()['AGE3_AMT']
df2_agg

CLST
0    690486.000000
1     46262.423792
2    257342.686667
Name: AGE3_AMT, dtype: float64

In [12]:
# Answer 2
print(np.min(df2_agg))
ans22 = np.round(np.min(df2_agg), 0)
print(ans22)

46262.42379182156
46262.0


### <span style='background-color:#fff651'> Question 3 </span>

In [13]:
# Step 3-1
df3_cat = purchase.pivot_table(index='USER_ID', columns='PRD_CAT', values='PUR_AMT', aggfunc= 'sum', fill_value = 0).reset_index().rename_axis(None, axis=1)
df3_cat['MAX_CAT'] = df3_cat.drop(columns='USER_ID').idxmax(axis=1)
df3_cat.columns = ['USER_ID','CAT1','CAT2','CAT3','PRF_CAT']
df3_cat.head()

,USER_ID,CAT1,CAT2,CAT3,PRF_CAT
0,1000003,162439,29054,10906,1
1,1000004,205987,0,0,1
2,1000005,104097,12841,10851,1
3,1000006,93457,35997,59314,1
4,1000007,158969,0,0,1


In [14]:
df3 = purchase.groupby(['USER_ID','GENDER','AGE_GRP','MARITAL_YN']).sum().reset_index().drop(columns='PRD_CAT')
df3 = pd.merge(left=df3, right=df3_cat[['USER_ID','PRF_CAT']], how='inner', on='USER_ID')
df3.rename(columns = {'PUR_AMT' : 'TOT_AMT'}, inplace = True)
df3.head()

,USER_ID,GENDER,AGE_GRP,MARITAL_YN,TOT_AMT,PRF_CAT
0,1000003,M,26-35,0,202399,1
1,1000004,M,46-50,1,205987,1
2,1000005,M,26-35,1,127789,1
3,1000006,F,51-55,0,188768,1
4,1000007,M,36-45,1,158969,1


In [15]:
# Step 3-2
q2 = df3.TOT_AMT.quantile(0.5)
q3 = df3.TOT_AMT.quantile(0.75)
df3_filtered = df3[(df3['TOT_AMT']>=q2)&(df3['TOT_AMT']<=q3)]
df3_filtered.shape

(321, 6)

In [16]:
# Step 3-3
df3_dummies = pd.get_dummies(df3_filtered[['GENDER','AGE_GRP','MARITAL_YN','PRF_CAT']].astype(str))
df3 = df3_filtered.drop(columns=['GENDER','AGE_GRP','MARITAL_YN','PRF_CAT']).join(df3_dummies)
df3 = df3.sort_values('USER_ID')
df3.head()

,USER_ID,TOT_AMT,GENDER_F,GENDER_M,AGE_GRP_18-25,AGE_GRP_26-35,AGE_GRP_36-45,AGE_GRP_46-50,AGE_GRP_51-55,MARITAL_YN_0,MARITAL_YN_1,PRF_CAT_1,PRF_CAT_3
0,1000003,202399,0,1,0,1,0,0,0,1,0,1,0
1,1000004,205987,0,1,0,0,0,1,0,0,1,1,0
3,1000006,188768,1,0,0,0,0,0,1,1,0,1,0
8,1000013,410999,0,1,0,0,0,1,0,0,1,1,0
10,1000015,242334,0,1,0,1,0,0,0,1,0,1,0


In [17]:
df3_filtered['PRF_CAT'].value_counts()

1    311
3     10
Name: PRF_CAT, dtype: int64

In [18]:
df3.columns

Index(['USER_ID', 'TOT_AMT', 'GENDER_F', 'GENDER_M', 'AGE_GRP_18-25',
       'AGE_GRP_26-35', 'AGE_GRP_36-45', 'AGE_GRP_46-50', 'AGE_GRP_51-55',
       'MARITAL_YN_0', 'MARITAL_YN_1', 'PRF_CAT_1', 'PRF_CAT_3'],
      dtype='object')

In [19]:
# Step 3-4
train3 = df3[df3['USER_ID']%4!=0].drop(columns='USER_ID')
test3 = df3[df3['USER_ID']%4==0].drop(columns='USER_ID')
print(train3.shape, test3.shape)

(248, 12) (73, 12)


In [20]:
# Step 3-5
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state=1234, max_depth=6)
dtr.fit(train3.drop(columns='TOT_AMT'), train3.TOT_AMT)

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1234, splitter='best')

In [18]:
# Step 23-6
y = dtr.predict(test3.drop(columns='TOT_AMT'))
np.mean(abs((y-test3.TOT_AMT)/test3.TOT_AMT))*100

26.53749944749592

In [19]:
# Answer 23
ans23 = np.round(np.mean(abs((y-test3.TOT_AMT)/test3.TOT_AMT))*100, 2)
ans23                 

26.54